#LAB 2

Working with big query
Remember you can use ! bq ' stuf...
Start by creating the needed tasks in bq then come back?
Using dataset
gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv


In [67]:
# pip installs
! pip install --upgrad google-cloud-bigquery




Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

ambiguous option: --upgrad (--upgrade, --upgrade-strategy?)


In [ ]:
#Copy Over data
# gsutil cp gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv ~/
# for whatever reason this doesn't work in notebook. Open cli and paste there.

In [1]:
from google.cloud import bigquery
import pandas as pd


In [2]:
project_id = "qwiklabs-gcp-01-14276dd314fb"
location =  "us-central1"


In [3]:
# make dataset and table
!bq mk aurora_data
!bq ls
!bq load \
  --autodetect \
  --source_format=CSV \
  --skip_leading_rows=1 \
  aurora_data.faqs_table \
  ./aurora-bay-faqs.csv



BigQuery error in mk operation: Dataset 'qwiklabs-
gcp-01-14276dd314fb:aurora_data' already exists.
   datasetId   
 ------------- 
  aurora_data  
  dataset      
Upload complete.
Waiting on bqjob_r26bf0133a5a885f2_0000019ae1e5e544_1 ... (1s) Current status: DONE   


In [ ]:
!bq ls aurora_data

In [4]:
mydataset = "aurora_data"
mymodel = "mymodel"
mytable = "faqs_table"
client = bigquery.Client(project=project_id)

In [5]:
## Makes a model that is connected to BigQuerry...
makeModel = f"""CREATE OR REPLACE MODEL `{project_id}.{mydataset}.{mymodel}`
REMOTE WITH CONNECTION DEFAULT
OPTIONS (ENDPOINT = 'text-embedding-005');"""

res = client.query(makeModel)

Generate embeddings for each question answere pair

In [6]:
##
%%bigquery df --project {project_id}
  SELECT *
  FROM aurora_data.faqs_table

Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
df[:5]

,string_field_0,string_field_1
0,When was Aurora Bay founded?,Aurora Bay was founded in 1901 by a group of f...
1,What is the population of Aurora Bay?,Aurora Bay has a population of approximately 3...
2,Where is the Aurora Bay Town Hall located?,The Town Hall is located at 100 Harbor View Ro...
3,Who is the current mayor of Aurora Bay?,"The current mayor is Linda Greenwood, elected ..."
4,What are the primary industries in Aurora Bay?,The primary industries include commercial fish...


In [13]:

embedding = f"""
CREATE OR REPLACE TABLE`{project_id}.{mydataset}.withEmbedding` AS
  SELECT *
  FROM ML.GENERATE_EMBEDDING(
  MODEL `{project_id}.{mydataset}.{mymodel}`,
  (
    SELECT string_field_0, string_field_1,	CONCAT(string_field_0, string_field_1) AS content
    FROM `{project_id}.{mydataset}.{mytable}`)
  );
"""
res = client.query(embedding)
#print (embedding)

In [14]:
%%bigquery df --project {project_id}
  SELECT *
  FROM aurora_data.withEmbedding

Query is running:   0%|          |

Downloading:   0%|          |

In [15]:
df[:5]

,ml_generate_embedding_result,ml_generate_embedding_statistics,ml_generate_embedding_status,string_field_0,string_field_1,content
0,"[-0.02901100181043148, 0.013011284172534943, 0...","{""token_count"":29,""truncated"":false}",,What are the primary industries in Aurora Bay?,The primary industries include commercial fish...,What are the primary industries in Aurora Bay?...
1,"[-0.02901100181043148, 0.013011284172534943, 0...","{""token_count"":29,""truncated"":false}",,What are the primary industries in Aurora Bay?,The primary industries include commercial fish...,What are the primary industries in Aurora Bay?...
2,"[-0.08013758808374405, -0.03401567041873932, -...","{""token_count"":29,""truncated"":false}",,Who is the current mayor of Aurora Bay?,"The current mayor is Linda Greenwood, elected ...",Who is the current mayor of Aurora Bay?The cur...
3,"[-0.08013758808374405, -0.03401567041873932, -...","{""token_count"":29,""truncated"":false}",,Who is the current mayor of Aurora Bay?,"The current mayor is Linda Greenwood, elected ...",Who is the current mayor of Aurora Bay?The cur...
4,"[-0.0732235312461853, -0.04528098553419113, -0...","{""token_count"":31,""truncated"":false}",,Does Aurora Bay have a police department?,Yes. The Aurora Bay Police Department is locat...,Does Aurora Bay have a police department?Yes. ...


In [16]:
!bq ls {mydataset}

     tableId      Type    Labels   Time Partitioning   Clustered Fields  
 --------------- ------- -------- ------------------- ------------------ 
  faqs_table      TABLE                                                  
  withEmbedding   TABLE                                                  


Bot can use VECTOR_SEARCH to find an embedding that matches the input




In [20]:
question = "Who is the mayor?"
answere = "string_field_1"

instructions = f"""
  SELECT {answere}
  FROM VECTOR_SEARCH(
    TABLE `{project_id}.{mydataset}.{mytable}`,
    'ml_generate_embedding_result',
    (
    SELECT ml_generate_embedding_result, '{question}' AS query
    FROM ML.GENERATE_EMBEDDING(
      MODEL `{project_id}.{mydataset}.withEmbeddings`,
      (SELECT {question} AS content))),
  top_k => 5
  """
print(instructions)


  SELECT string_field_1	
  FROM VECTOR_SEARCH(
    TABLE `qwiklabs-gcp-01-14276dd314fb.aurora_data.faqs_table`,
    'ml_generate_embedding_result',
    (
    SELECT ml_generate_embedding_result, 'Who is the mayor?' AS query
    FROM ML.GENERATE_EMBEDDING(
      MODEL `qwiklabs-gcp-01-14276dd314fb.aurora_data.withEmbeddings`,
      (SELECT Who is the mayor? AS content))),
  top_k => 5
  


In [17]:
res = client.query(instructions)


QueryJob<project=qwiklabs-gcp-01-14276dd314fb, location=US, id=d94bbb25-5154-4550-8d37-78ac3256e50b>


In [18]:
print(res.results)

AttributeError: 'QueryJob' object has no attribute 'results'